# Analysing data

- We have data loaded into Python data structures
- What can we learn from the data?
- How do we learn from the data?

## Tools

- [Python's operator module](https://docs.python.org/3/library/operator.html)
- [Python's max builtin](https://docs.python.org/3/library/functions.html#max)
- [Python string formatting](https://docs.python.org/3.1/library/string.html#format-string-syntax)

## Examples

We'll walk through a very simple example, building on concepts from our previous step:

- request a CSV file, and load the data we extract
- clean the data a little for easier usage
- run a number of summary queries over the data

In [44]:
# just for presentation in notebooks
from pprint import pprint as print

## Request

Identify and download a dataset of interest.

In [45]:
import operator
import requests
import csv


csv_source = 'http://data.defra.gov.uk/env/strategic_noise_mapping/r2_strategic_noise_mapping.csv'

csv_delimiter = ','

response = requests.get(csv_source)

raw = response.text.splitlines()

reader = csv.DictReader(raw, delimiter=csv_delimiter)

## Extract and clean

As we read the data into a Python data structure, we'll also do some basic cleaning.

In [46]:
def clean(row):
    """Clean rows of data."""
    
    # remove a noisy row from our data - it is not about a specific agglomeration
    if row['Location/Agglomeration'] == 'Major sources (outside agglomerations)':
        row = None
    else:
        for key, value in row.items():
            # some of the population counts show 'n/a'.
            if value == 'n/a':
                row[key] = 0
            # when a value can be coerced to an integer, then do it.
            try:
                row[key] = int(value)
            except ValueError as e:
                pass
    return row

In [47]:
data = []

for row in reader:
    row = clean(row)
    if row:
        data.append(row)

## A sample

In [48]:
print(data[0])

{'AgglomerationPopulation': 895000,
 'Industry_Pop_Lden>=55dB': 1400,
 'Industry_Pop_Lden>=60dB': 600,
 'Industry_Pop_Lden>=65dB': 100,
 'Industry_Pop_Lden>=70dB': 0,
 'Industry_Pop_Lden>=75dB': 0,
 'Industry_Pop_Lnight>=50dB': 1100,
 'Industry_Pop_Lnight>=55dB': 400,
 'Industry_Pop_Lnight>=60dB': 100,
 'Industry_Pop_Lnight>=65dB': 0,
 'Industry_Pop_Lnight>=70dB': 0,
 'Location/Agglomeration': 'Tyneside',
 'Railways_Pop_Lden>=55dB': 14200,
 'Railways_Pop_Lden>=60dB': 8100,
 'Railways_Pop_Lden>=65dB': 3900,
 'Railways_Pop_Lden>=70dB': 1700,
 'Railways_Pop_Lden>=75dB': 200,
 'Railways_Pop_Lnight>=50dB': 10400,
 'Railways_Pop_Lnight>=55dB': 6000,
 'Railways_Pop_Lnight>=60dB': 2500,
 'Railways_Pop_Lnight>=65dB': 1100,
 'Railways_Pop_Lnight>=70dB': 0,
 'Road_Pop_Lden>=55dB': 166400,
 'Road_Pop_Lden>=60dB': 79200,
 'Road_Pop_Lden>=65dB': 46100,
 'Road_Pop_Lden>=70dB': 18200,
 'Road_Pop_Lden>=75dB': 1300,
 'Road_Pop_Lnight>=50dB': 94800,
 'Road_Pop_Lnight>=55dB': 51600,
 'Road_Pop_Lnight>=60d

##  Queries

What high-level information, does this dataset hold?

In [49]:
# high-level data points
columns = len(data[0].keys())

rows = len(data)

most_populated = max(data, key=operator.itemgetter('AgglomerationPopulation'))

least_populated = min(data, key=operator.itemgetter('AgglomerationPopulation'))

print(most_populated)

print(least_populated)

{'AgglomerationPopulation': 9300000,
 'Industry_Pop_Lden>=55dB': 23600,
 'Industry_Pop_Lden>=60dB': 13000,
 'Industry_Pop_Lden>=65dB': 7500,
 'Industry_Pop_Lden>=70dB': 4600,
 'Industry_Pop_Lden>=75dB': 3000,
 'Industry_Pop_Lnight>=50dB': 20500,
 'Industry_Pop_Lnight>=55dB': 11300,
 'Industry_Pop_Lnight>=60dB': 6700,
 'Industry_Pop_Lnight>=65dB': 4000,
 'Industry_Pop_Lnight>=70dB': 2700,
 'Location/Agglomeration': 'Greater London Urban Area',
 'Railways_Pop_Lden>=55dB': 525200,
 'Railways_Pop_Lden>=60dB': 308500,
 'Railways_Pop_Lden>=65dB': 158100,
 'Railways_Pop_Lden>=70dB': 59800,
 'Railways_Pop_Lden>=75dB': 15200,
 'Railways_Pop_Lnight>=50dB': 388700,
 'Railways_Pop_Lnight>=55dB': 214200,
 'Railways_Pop_Lnight>=60dB': 95100,
 'Railways_Pop_Lnight>=65dB': 29700,
 'Railways_Pop_Lnight>=70dB': 6400,
 'Road_Pop_Lden>=55dB': 2387200,
 'Road_Pop_Lden>=60dB': 1426100,
 'Road_Pop_Lden>=65dB': 1027200,
 'Road_Pop_Lden>=70dB': 597800,
 'Road_Pop_Lden>=75dB': 99200,
 'Road_Pop_Lnight>=50dB': 1

In [50]:
most_populated_high_exposure_count = sum([
    most_populated['Industry_Pop_Lden>=75dB'], 
    most_populated['Railways_Pop_Lden>=75dB'], 
    most_populated['Road_Pop_Lden>=75dB']
])

most_populated_high_exposure_percent = '{0:.0f}%'.format(
    (most_populated_high_exposure_count / most_populated['AgglomerationPopulation']) * 100
)

least_populated_high_exposure_count = sum([
    least_populated['Industry_Pop_Lden>=75dB'], 
    least_populated['Railways_Pop_Lden>=75dB'], 
    least_populated['Road_Pop_Lden>=75dB']
])

least_populated_high_exposure_percent = '{0:.0f}%'.format(
    (least_populated_high_exposure_count / least_populated['AgglomerationPopulation']) * 100
)

# A factual statement, according to this data source.
statement = """\
The data holds {columns} columns of data for {rows} different "Agglomerations". \
 \
The most populated "Agglomeration" is "{most_populated_place_name}" with a population of {most_populated_pop_count}. \
Out of this population, {most_populated_high_exposure_count} ({most_populated_high_exposure_percent}) \
people are exposed to very high levels of noise pollution from industry, railway and road sources.\
 \
The least populated "Agglomeration" is "{least_populated_place_name}" with a population of {least_populated_pop_count}. \
Out of this population, {least_populated_high_exposure_count} ({least_populated_high_exposure_percent}) \
people are exposed to very high levels of noise pollution from industry, railway and road sources.\
""".format(
    columns=columns, 
    rows=rows, 
    most_populated_place_name=most_populated['Location/Agglomeration'], 
    most_populated_pop_count=most_populated['AgglomerationPopulation'],
    most_populated_high_exposure_count=most_populated_high_exposure_count,
    most_populated_high_exposure_percent=most_populated_high_exposure_percent,
    least_populated_place_name=least_populated['Location/Agglomeration'], 
    least_populated_pop_count=least_populated['AgglomerationPopulation'], 
    least_populated_high_exposure_count=least_populated_high_exposure_count,
    least_populated_high_exposure_percent=least_populated_high_exposure_percent
)

print(statement)

('The data holds 32 columns of data for 65 different "Agglomerations".  The '
 'most populated "Agglomeration" is "Greater London Urban Area" with a '
 'population of 9300000. Out of this population, 117400 (1%) people are '
 'exposed to very high levels of noise pollution from industry, railway and '
 'road sources. The least populated "Agglomeration" is "Chesterfield/Staveley" '
 'with a population of 105000. Out of this population, 200 (0%) people are '
 'exposed to very high levels of noise pollution from industry, railway and '
 'road sources.')
